<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Import-dataset" data-toc-modified-id="Import-dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import dataset</a></span></li><li><span><a href="#Sentence-embedding" data-toc-modified-id="Sentence-embedding-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Sentence embedding</a></span><ul class="toc-item"><li><span><a href="#Infersent" data-toc-modified-id="Infersent-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Infersent</a></span></li></ul></li><li><span><a href="#Dimensionality-reduction" data-toc-modified-id="Dimensionality-reduction-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Dimensionality reduction</a></span><ul class="toc-item"><li><span><a href="#PCA" data-toc-modified-id="PCA-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>PCA</a></span></li></ul></li><li><span><a href="#Blocking" data-toc-modified-id="Blocking-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Blocking</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Evaluation</a></span><ul class="toc-item"><li><span><a href="#bilstm,-rnn_dim-2048" data-toc-modified-id="bilstm,-rnn_dim-2048-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>bilstm, rnn_dim 2048</a></span></li><li><span><a href="#Searching" data-toc-modified-id="Searching-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Searching</a></span></li></ul></li><li><span><a href="#Result" data-toc-modified-id="Result-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Result</a></span></li></ul></div>

# Setup 

In [1]:
import os
import sys
import time
import numpy as np
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
params = {}

# Import dataset

In [2]:
from preprocessing_datasets.preprocessing_abt_buy import clean_abt_buy
table, pairs = clean_abt_buy()

In [3]:
table

,name,description,price
0,Sony Turntable - PSLX350H,Sony Turntable - PSLX350H/ Belt Drive System/ ...,unk
1,Bose Acoustimass 5 Series III Speaker System -...,Bose Acoustimass 5 Series III Speaker System -...,$399.00
2,Sony Switcher - SBV40S,Sony Switcher - SBV40S/ Eliminates Disconnecti...,$49.00
3,Sony 5 Disc CD Player - CDPCE375,Sony 5 Disc CD Player- CDPCE375/ 5 Disc Change...,unk
4,Bose 27028 161 Bookshelf Pair Speakers In Whit...,Bose 161 Bookshelf Speakers In White - 161WH/ ...,$158.00
...,...,...,...
2168,Sony VAIO FW378J/B Notebook - VGNFW378J/B,Intel Centrino 2 Core 2 Duo P8600 2.4GHz - 16....,unk
2169,Sennheiser CX380 Sennheiser CX 380 Sport II Gr...,unk,unk
2170,IWORK 09 RETAIL-INT DVD - MB942Z/A,unk,unk
2171,IWORK 09 FAMILY PACK-INT DVD - MB943Z/A,unk,unk


# Sentence embedding

## Infersent

Setup model

In [4]:
from embedding_algorithms.inferSent import set_RNN_embedding
start = time.time()
model_type = "bilstm" 
char_level = False
model_version = 2
rnn_dim = 2048
verbose = 1
set_RNN_embedding(model_type, char_level, model_version, rnn_dim, verbose)
print("TIME: {0}".format(time.time() - start))

params['embedding'] = {
    'name': 'Infersent',
    'model_type': model_type,
    'char_level': char_level,
    'model_version': model_version,
    'rnn_dim': rnn_dim,
    'verbose': verbose
}

Vocab size : 2196017
TIME: 176.14648389816284


Embedding

In [5]:
from embedding_algorithms.inferSent import RNN_embedding
start = time.time()
attr_list = ['name']
params['attr_list'] = attr_list
embeddings_tokens = RNN_embedding(table, attr_list, model_type, char_level)
embeddings_tokens = np.array(embeddings_tokens)
print("TIME: {0}".format(time.time() - start))

attrs: ['name']
TIME: 36.75310301780701


# Dimensionality reduction

## PCA

In [6]:
from dimensionality_reduction_algorithms.pca import pca_dim_reduction

start = time.time()
pca_embeddings = pca_dim_reduction(
            embeddings_tokens, 
            num_components=2,
            verbose=1)

print("PCA: {0}".format(time.time() - start))

params['reduction'] = {
    'name': 'PCA',
    'num_components': 2,
    'verbose': 1,
}

starting dimension: 4096
setting PCA with n_components: 2
PCA: 0.3854849338531494


# Blocking

In [11]:
from cluster_algorithms.hierarchy_cluster import hierarchy_cluster
num_clusters = 3
start = time.time()
blocks = hierarchy_cluster(pca_embeddings, {'num_clusters': num_clusters})
print("BLOCKS: {0}".format(time.time() - start))
params['blocking'] = {
    'name': 'hierarchy',
    'num_clusters': num_clusters
}

clustering with NUM_CLUSTERS = 3, 
BLOCKS: 0.13967204093933105


# Evaluation

## bilstm, rnn_dim 2048

In [12]:
from evaluation import calc_index
calc_index(blocks,table,pairs)
print()
print(params['attr_list'])
print(params['embedding'])
print(params['reduction'])
print(params['blocking'])

(RR) Reduction ratio is: 0.6127571001551775
(PC) Pair completeness is: 0.6007292616226071
(RM) Reference metric (Harmonic mean RR and PC) is: 0.6066835721020437
(PQ) Pair quality - Precision is: 0.0007211280675299777
(FM) Fmeasure is: 0.001440526896211021

['name']
{'name': 'Infersent', 'model_type': 'bilstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 2048, 'verbose': 1}
{'name': 'PCA', 'num_components': 2, 'verbose': 1}
{'name': 'hierarchy', 'num_clusters': 3}


## Searching

In [10]:
for i in range(1,10,1):
    num_clusters = int(i)
    blocks = hierarchy_cluster(pca_embeddings, {'num_clusters': num_clusters})
    calc_index(blocks,table,pairs)

clustering with NUM_CLUSTERS = 1, 
(RR) Reduction ratio is: 0.0
(PC) Pair completeness is: 1.0
(RM) Reference metric (Harmonic mean RR and PC) is: 0
(PQ) Pair quality - Precision is: 0.0004648545390905801
(FM) Fmeasure is: 0.0009292770995033831
clustering with NUM_CLUSTERS = 2, 
(RR) Reduction ratio is: 0.4998614335147834
(PC) Pair completeness is: 0.6235186873290793
(RM) Reference metric (Harmonic mean RR and PC) is: 0.5548842089843046
(PQ) Pair quality - Precision is: 0.0005795303770505971
(FM) Fmeasure is: 0.0011579844637084453
clustering with NUM_CLUSTERS = 3, 
(RR) Reduction ratio is: 0.6127571001551775
(PC) Pair completeness is: 0.6007292616226071
(RM) Reference metric (Harmonic mean RR and PC) is: 0.6066835721020437
(PQ) Pair quality - Precision is: 0.0007211280675299777
(FM) Fmeasure is: 0.001440526896211021
clustering with NUM_CLUSTERS = 4, 
(RR) Reduction ratio is: 0.7310920310287227
(PC) Pair completeness is: 0.48678213309024615
(RM) Reference metric (Harmonic mean RR and PC

# Result

In [28]:
# RR PC alpha

list_hierarchy_tsne = [0.9078, 0.9134, 0.9106] 
list_hierarchy_pca = [0.6127, 0.6007, 0.6066] # PCA: 0.38, BLOCKS: 0.139, n_cluster: 3
